In [1]:
from utils import sheet_to_dataframe, convert_islamic_date
import pandas as pd
import numpy as np
import warnings
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


warnings.simplefilter('ignore')
pd.pandas.set_option('display.max_columns',None)

/home/bee_nyah/Desktop/DATA_EPIC/Best_Nyah/venv/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# read data from google sheet
WORKSHEETSHEET_NAME="inconsistent_data"
SHEET_URL = "https://docs.google.com/spreadsheets/d/1SUlcukpgxf6pfFQbj6DKQZXSlOhyBOSuBdbmQ2ZbhTM/edit#gid=1520648174"

data = sheet_to_dataframe(SHEET_URL,WORKSHEETSHEET_NAME)

In [3]:
data.head()

,S#,Date,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,Location,Location Category,Location Sensitivity,Open/Closed Space,Influencing Event/Event,Target Type,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)
0,1,Sunday-November 19-1995,25 Jumaada al-THaany 1416 A.H,Holiday,Weekend,N/A,Islamabad,33.718,73.0718,Capital,Egyptian Embassy,Foreign,High,Closed,,Foreigner,None,14,15,,60,2,,,15.835,60.503
1,2,Monday-November 6-2000,10 SHa`baan 1421 A.H,Working Day,,N/A,Karachi,24.9918,66.9911,Sindh,office of Nawa-e-Waqt,Office Building,Low,Closed,,Media,None,,3,,3,1,,,23.77,74.786
2,3,Wednesday-May 8-2002,25 safar 1423 A.H,Working Day,,7:45 AM,Karachi,24.9918,66.9911,Sindh,Pakistan Navy bus Parked outside Five Star She...,Hotel,Medium,Closed,,Foreigner,Christian,13,15,20,40,1,2.5 Kg,1.Jinnah Postgraduate Medical Center 2. Civil ...,31.46,88.628
3,4,Friday-June 14-2002,3 Raby` al-THaany 1423 A.H,Working Day,,11:10:00 AM,Karachi,24.9918,66.9911,Sindh,US Consulate Civil Lines Area,Foreign,High,Closed,,Foreigner,Christian,,12,,51,1,NA,NA,31.43,88.574
4,5,Friday-July 4-2003,4 Jumaada al-awal 1424 A.H,Working Day,,N/A,Quetta,30.2095,67.0182,Baluchistan,Imambargah MeCongy Road Quetta,Religious,Medium,Closed,during Friday prayer,Religious,Shiite,44,47,,65,1,NA,1.CMH Quetta \n2.Civil Hospital 3. Boland Medi...,33.12,91.616


In [4]:
data.shape

(496, 26)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   S#                       496 non-null    int64 
 1   Date                     496 non-null    object
 2   Islamic Date             496 non-null    object
 3   Blast Day Type           496 non-null    object
 4   Holiday Type             496 non-null    object
 5   Time                     496 non-null    object
 6   City                     496 non-null    object
 7   Latitude                 496 non-null    object
 8   Longitude                496 non-null    object
 9   Province                 496 non-null    object
 10  Location                 496 non-null    object
 11  Location Category        496 non-null    object
 12  Location Sensitivity     496 non-null    object
 13  Open/Closed Space        496 non-null    object
 14  Influencing Event/Event  496 non-null    o

In [6]:
df = data.copy()

In [7]:
df.replace('', np.nan, inplace=True)
df.replace('N/A', np.nan, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   S#                       496 non-null    int64  
 1   Date                     496 non-null    object 
 2   Islamic Date             342 non-null    object 
 3   Blast Day Type           486 non-null    object 
 4   Holiday Type             72 non-null     object 
 5   Time                     350 non-null    object 
 6   City                     496 non-null    object 
 7   Latitude                 493 non-null    float64
 8   Longitude                493 non-null    float64
 9   Province                 496 non-null    object 
 10  Location                 493 non-null    object 
 11  Location Category        460 non-null    object 
 12  Location Sensitivity     460 non-null    object 
 13  Open/Closed Space        461 non-null    object 
 14  Influencing Event/Event  2

In [8]:
# data types
data_type_mapping = {
    'Latitude': 'float64',
    'Temperature(C)': 'float64',
    'Temperature(F)': 'float64',
    'Killed Min': 'int64',
    'Killed Max': 'int64',
    'Injured Min': 'int64',
    'No. of Suicide Blasts': 'int64',
}
			
for col in data_type_mapping:
        df[col].fillna(-59, inplace=True)

# convert respective columns to the right types
df = df.astype(data_type_mapping)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   S#                       496 non-null    int64  
 1   Date                     496 non-null    object 
 2   Islamic Date             342 non-null    object 
 3   Blast Day Type           486 non-null    object 
 4   Holiday Type             72 non-null     object 
 5   Time                     350 non-null    object 
 6   City                     496 non-null    object 
 7   Latitude                 496 non-null    float64
 8   Longitude                493 non-null    float64
 9   Province                 496 non-null    object 
 10  Location                 493 non-null    object 
 11  Location Category        460 non-null    object 
 12  Location Sensitivity     460 non-null    object 
 13  Open/Closed Space        461 non-null    object 
 14  Influencing Event/Event  2

In [10]:
df.head()

,S#,Date,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,Location,Location Category,Location Sensitivity,Open/Closed Space,Influencing Event/Event,Target Type,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)
0,1,Sunday-November 19-1995,25 Jumaada al-THaany 1416 A.H,Holiday,Weekend,NaN,Islamabad,33.7180,73.0718,Capital,Egyptian Embassy,Foreign,High,Closed,NaN,Foreigner,None,14,15,-59,60,2,NaN,NaN,15.835,60.503
1,2,Monday-November 6-2000,10 SHa`baan 1421 A.H,Working Day,NaN,NaN,Karachi,24.9918,66.9911,Sindh,office of Nawa-e-Waqt,Office Building,Low,Closed,NaN,Media,None,-59,3,-59,3,1,NaN,NaN,23.770,74.786
2,3,Wednesday-May 8-2002,25 safar 1423 A.H,Working Day,NaN,7:45 AM,Karachi,24.9918,66.9911,Sindh,Pakistan Navy bus Parked outside Five Star She...,Hotel,Medium,Closed,NaN,Foreigner,Christian,13,15,20,40,1,2.5 Kg,1.Jinnah Postgraduate Medical Center 2. Civil ...,31.460,88.628
3,4,Friday-June 14-2002,3 Raby` al-THaany 1423 A.H,Working Day,NaN,11:10:00 AM,Karachi,24.9918,66.9911,Sindh,US Consulate Civil Lines Area,Foreign,High,Closed,NaN,Foreigner,Christian,-59,12,-59,51,1,NA,NA,31.430,88.574
4,5,Friday-July 4-2003,4 Jumaada al-awal 1424 A.H,Working Day,NaN,NaN,Quetta,30.2095,67.0182,Baluchistan,Imambargah MeCongy Road Quetta,Religious,Medium,Closed,during Friday prayer,Religious,Shiite,44,47,-59,65,1,NA,1.CMH Quetta \n2.Civil Hospital 3. Boland Medi...,33.120,91.616


In [11]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')	

In [12]:
# Remove 'kg' suffix from the 'Explosive Weight (max)' column
df['Explosive Weight (max)'] = df['Explosive Weight (max)'].str.replace(r'[kK][gG]$', '', regex=True)
# Remove '+' from the 'Injured max' column
df["Injured Max"] = df["Injured Max"].str.replace(r'[A-Za-z+]', '', regex=True)

In [26]:
df['Islamic Date'].fillna('Unknown', inplace=True)
df['Holiday Type'].fillna('Unknown', inplace=True)
df['Time'].fillna('Unknown', inplace=True)
df['Hospital Names'].fillna('Unknown', inplace=True)
df['Explosive Weight (max)'].fillna('Unknown', inplace=True)
df['Blast Day Type'].fillna('Unknown', inplace=True)
df['Islamic Date'].fillna('Unknown', inplace=True)
df['Latitude'].fillna('Unknown', inplace=True)
df['Longitude'].fillna('Unknown', inplace=True) 

In [23]:
# using fuzzy matching for location column 
def standardize_location(location_name):
    choices = df['Location'].unique()
    match, score = process.extractOne(location_name, choices, scorer=fuzz.token_sort_ratio)
    if score >= 80:
        return match
    else:
        return location_name

df['Location'] = df['Location'].astype(str)
df['Location'] = df['Location'].apply(standardize_location)

In [29]:
df['Location'].head()

0                                     Egyptian Embassy
1                                office of Nawa-e-Waqt
2    Pakistan Navy bus Parked outside Five Star She...
3                        US Consulate Civil Lines Area
4                       Imambargah MeCongy Road Quetta
Name: Location, dtype: object

In [28]:
df.head()

,S#,Date,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,Location,Location Category,Location Sensitivity,Open/Closed Space,Influencing Event/Event,Target Type,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)
0,1,1995-11-19,25 Jumaada al-THaany 1416 A.H,Holiday,Weekend,Unknown,Islamabad,33.7180,73.0718,Capital,Egyptian Embassy,Foreign,High,Closed,NaN,Foreigner,None,14,15,-59,NaN,2,Unknown,Unknown,15.835,60.503
1,2,2000-11-06,10 SHa`baan 1421 A.H,Working Day,Unknown,Unknown,Karachi,24.9918,66.9911,Sindh,office of Nawa-e-Waqt,Office Building,Low,Closed,NaN,Media,None,-59,3,-59,NaN,1,Unknown,Unknown,23.770,74.786
2,3,2002-05-08,25 safar 1423 A.H,Working Day,Unknown,7:45 AM,Karachi,24.9918,66.9911,Sindh,Pakistan Navy bus Parked outside Five Star She...,Hotel,Medium,Closed,NaN,Foreigner,Christian,13,15,20,NaN,1,2.5,1.Jinnah Postgraduate Medical Center 2. Civil ...,31.460,88.628
3,4,2002-06-14,3 Raby` al-THaany 1423 A.H,Working Day,Unknown,11:10:00 AM,Karachi,24.9918,66.9911,Sindh,US Consulate Civil Lines Area,Foreign,High,Closed,NaN,Foreigner,Christian,-59,12,-59,NaN,1,NA,NA,31.430,88.574
4,5,2003-07-04,4 Jumaada al-awal 1424 A.H,Working Day,Unknown,Unknown,Quetta,30.2095,67.0182,Baluchistan,Imambargah MeCongy Road Quetta,Religious,Medium,Closed,during Friday prayer,Religious,Shiite,44,47,-59,NaN,1,NA,1.CMH Quetta \n2.Civil Hospital 3. Boland Medi...,33.120,91.616
